In [81]:
import json
from collections import defaultdict

# Function to convert time format to the desired schema
def convert_time_to_schema(time_str):
    # Check if the time is in standard "HH:MM AM/PM" format
    if isinstance(time_str, str) and ' ' in time_str:
        time, clock = time_str.split(' ')
        return {"time": time, "clock": clock, "notification": True}
    # If it's a numerical value (e.g., 0.6666666667), try to handle it
    elif isinstance(time_str, (int, float)):
        # You can add custom handling for numbers here if needed
        # For example, converting 0.6666666667 to a standard time, or just defaulting to None.
        return {"time": str(time_str), "clock": "", "notification": False}
    else:
        # In case of unexpected format, return a default value or skip it
        return {"time": "", "clock": "", "notification": False}

# Function to process the input data and generate the new schema
def process_prayer_times(input_file, output_file):
    # Read the input data from the JSON file
    with open(input_file, 'r') as infile:
        data = json.load(infile)

    # Prepare the output structure
    output_data = defaultdict(dict)

    # Iterate over the months and dates
    for month, entries in data.items():
        for entry in entries:
            date_str = entry["Date"]
            day = int(date_str.split()[0])  # Extract the day number from "01 November 2024"
            date = str(day)

            # Build the structure for each prayer time
            output_data[month][date] = {
                "Fajr": {
                    "Adhaan": convert_time_to_schema(entry["Fajr Adhaan"]),
                    "Iqamah": convert_time_to_schema(entry["Fajr Iqamah"]),
                },
                "Dhuhr": {
                    "Adhaan": convert_time_to_schema(entry["Dhuhr Adhaan"]),
                    "Iqamah": convert_time_to_schema(entry["Dhuhr Iqamah"]),
                },
                "Asr": {
                    "Adhaan": convert_time_to_schema(entry["Asr Adhaan"]),
                    "Iqamah": convert_time_to_schema(entry["Asr Iqamah"]),
                },
                "Maghrib": {
                    "Adhaan": convert_time_to_schema(entry["Maghrib Adhaan"]),
                    "Iqamah": convert_time_to_schema(entry["Maghrib Iqamah"]),
                },
                "Isha": {
                    "Adhaan": convert_time_to_schema(entry["Isha Adhaan"]),
                    "Iqamah": convert_time_to_schema(entry["Isha Iqamah"]),
                }
            }

    # Write the transformed data into the output JSON file
    with open(output_file, 'w') as outfile:
        json.dump(output_data, outfile, indent=4)

# Example usage
input_file = 'prayer_time.json'  # Input JSON file
output_file = 'prayer_time_transformed.json'  # Output JSON file

process_prayer_times(input_file, output_file)
